# Stokes Equation (2D)

## Falling Block (constant viscosity, time-dependent)

We now turn to the **time-dependent**, isoviscous problem of the falling block.  

To solve this problem, we must couple the solution of the **momentum conservation** equation with the solution of the **advection equation**. The coupling is implemented by first solving the momentum conservation equation to obtain the velocities, and then transporting the density (or, in the case of passive markers, the phase) using the advection equation.

### The Problem

Unlike the full benchmark for this problem, we focus here only on the **isoviscous** case (the benchmark considers the sinking velocity and deformation of the block as a function of the viscosity contrast [see here](../../examples/Benchmarks/)).  

We assume a square body with a given width (**W**), height (**T**), and density ($\rho_b$), embedded in a viscous medium ($\eta_m$) with density ($\rho_m$). The modeled domain is also square, with a length of $L = 500\ \textrm{km}$ and a height of $H = 500\ \textrm{km}$. Throughout the domain, we apply *free-slip* velocity boundary conditions.

<img src="../Figures/Exercise09_1.png" alt="drawing" width="450"/> <br>
**Figure 1.** Model setup

A detailed description of how the model domain is discretized into a numerical grid, as well as the discretization of the **momentum conservation** and **mass conservation** equations depending on *free-slip* or *no-slip* boundary conditions, is provided [here](../../examples/StokesEquation/2D/README.md). Due to the length of this material, we omit a detailed explanation at this point.  

We would like to incorporate as many advection mechanisms as possible into this problem. Therefore, in addition to the modifications required for time dependence, we introduce further changes. These changes are highlighted in **bold** throughout the text.  

For additional information, see the [previous exercise](09_2D_Falling_Block.ipynb).

First, we load the necessary modules for visualizing results, assembling and solving the linear system, setting up initial conditions, and solving the two-dimensional momentum conservation problem.  
Because we now include advection and tracers, a few additional modules are required as well.

In [ ]:
using Plots
using ExtendableSparse
using GeoModBox.InitialCondition, GeoModBox.MomentumEquation.TwoD
using GeoModBox.AdvectionEquation.TwoD
using GeoModBox.Tracers.TwoD
using Base.Threads
using Printf

First, we define the **advection method** using the tuple `FD`.

In [ ]:
# Define Numerical Scheme =========================================== #
# Advection ---
#   1) upwind, 2) slf, 3) semilag, 4) tracers
FD          =   (Method     = (Adv=:tracers,),)
# ------------------------------------------------------------------- #

Using the built-in function `IniPhase()`, we can generate the initial density distribution for our *block* problem. Depending on the setup, this function creates an initial distribution of specific quantities on the numerical grid.  

To use it, we need the tuple `Ini`, which defines the distribution (here for the phase *p*). The density distribution created by `IniPhase()` on the centroids is used by all advection methods, **except for tracers**. In the tracer case, an alternative initialization is applied.  

In [ ]:
# Define Initial Condition ========================================== #
# Density --- 
#   1) block
Ini         =   (p=:block,) 
# ------------------------------------------------------------------- #
# Plot Settings ===================================================== #
Pl  =   (
    qinc    =   5,
    qsc     =   100*(60*60*24*365.25)*5e1
)
# ------------------------------------------------------------------- #

Next, we define the geometry of our model domain.  

In [ ]:
 # Geometry ========================================================== #
 M       =   (
    xmin    =   0.0,
    xmax    =   500.0e3,    # [ m ]
    ymin    =   -500.0e3,   # [ m ]
    ymax    =   0.0,
)
# ------------------------------------------------------------------- #

Now we define our grid resolution $\left(nc_x = nc_y = 50\right)$ and the numerical grid.  

We need to specify both the grid spacing $\Delta{x}$ and $\Delta{y}$, as well as the coordinates of the different grid points (*vertices* and *centroids*).  
To do this, we first define the respective 1-D coordinate vectors, from which the 2-D coordinate grids are then generated.  

In [ ]:
# Grid ============================================================== #
NC      =   (
    x   =   50, 
    y   =   50,
)
NV      =   (
    x   =   NC.x + 1,
    y   =   NC.y + 1,
)
Δ       =   (
    x   =   (M.xmax - M.xmin)/NC.x,
    y   =   (M.ymax - M.ymin)/NC.y,
)
x       =   (
    c   =   LinRange(M.xmin+Δ.x/2,M.xmax-Δ.x/2,NC.x),
    ce  =   LinRange(M.xmin - Δ.x/2.0, M.xmax + Δ.x/2.0, NC.x+2),
    v   =   LinRange(M.xmin,M.xmax,NV.x),
)
y       =   (
    c   =   LinRange(M.ymin+Δ.y/2,M.ymax-Δ.y/2,NC.y),
    ce  =   LinRange(M.ymin - Δ.x/2.0, M.ymax + Δ.x/2.0, NC.y+2),
    v   =   LinRange(M.ymin,M.ymax,NV.y),
)
x1      =   (
    c2d     =   x.c .+ 0*y.c',
    v2d     =   x.v .+ 0*y.v', 
    vx2d    =   x.v .+ 0*y.ce',
    vy2d    =   x.ce .+ 0*y.v',
)
x   =   merge(x,x1)
y1      =   (
    c2d     =   0*x.c .+ y.c',
    v2d     =   0*x.v .+ y.v',
    vx2d    =   0*x.v .+ y.ce',
    vy2d    =   0*x.ce .+ y.v',
)
y   =   merge(y,y1)
# ------------------------------------------------------------------- #

Next, we define the physical parameters: gravitational acceleration $g$, viscosity $\eta_0$ ($10^{21}$ Pa·s), background density $\rho_0$ (3200 kg/m³), and block density $\rho_1$ (3300 kg/m³).

The vector `phase` contains the phase indices, which are used by `IniPhase()` and by the marker initialization (`IniTracer2D()`) to assign the appropriate phase to each location.  
The vector $\rho$ stores the density values associated with each phase.

In [ ]:
# Physics =========================================================== #
g       =   9.81

η₀      =   1.0e21

ρ₀      =   3200.0          #   Background density
ρ₁      =   3300.0          #   Block density
ρ       =   [ρ₀,ρ₁] 

phase   =   [0,1]
# ------------------------------------------------------------------- #

For our time-dependent problem, we again define only the location for saving the animation.  


In [ ]:
# Animationsettings ================================================= #
path        =   string("./Results/")
anim        =   Plots.Animation(path, String[] )
filename    =   string("10_Falling_",Ini.p,"_iso_td_",FD.Method.Adv)
save_fig    =   1
# ------------------------------------------------------------------- #

Now we define the sizes of the required fields. **In this case, we also introduce some new fields.**  

In [ ]:
# Allocation ======================================================== #
D   =   (
    vx      =   zeros(Float64,NV.x,NC.y+2),
    vy      =   zeros(Float64,NC.x+2,NV.y),
    Pt      =   zeros(Float64,NC...),
    p       =   zeros(Float64,NC...),
    p_ex    =   zeros(Float64,NC.x+2,NC.y+2),
    ρ       =   zeros(Float64,NC...),
    ρ_ex    =   zeros(Float64,NC.x+2,NC.y+2),
    ρ_exo   =   zeros(Float64,NC.x+2,NC.y+2),
    vxc     =   zeros(Float64,NC...),
    vyc     =   zeros(Float64,NC...),
    vc      =   zeros(Float64,NC...),
    wt      =   zeros(Float64,(NC.x,NC.y)),
    wte     =   zeros(Float64,(NC.x+2,NC.y+2)),
    wtv     =   zeros(Float64,(NV...)),
    ηv      =   zeros(Float64,NV...),
)
# ------------------------------------------------------------------- #

Now we can define the boundary and initial conditions.  

For the boundary conditions, we again use the tuple `VBC`, which specifies the type of boundary (*free slip* or *no slip*) at each position (**E**, **W**, **S**, **N**).  
The value specified by `val` then sets the velocity at the respective boundary.  

In [ ]:
# Boundary Conditions =============================================== #
VBC     =   (
    type    =   (E=:freeslip,W=:freeslip,S=:freeslip,N=:freeslip),
    val     =   (E=zeros(NV.y),W=zeros(NV.y),S=zeros(NV.x),N=zeros(NV.x),
                vxE=zeros(NC.y),vxW=zeros(NC.y),vyS=zeros(NC.x),vyN=zeros(NC.x)),
)
# ------------------------------------------------------------------- #

Next, we need to define the parameters for time and the time step length.  

In [ ]:
# Time ============================================================== #
T   =   ( 
    tmax    =   [0.0],  
    Δfac    =   1.0,    # Courant time factor, i.e. dtfac*dt_courant
    Δ       =   [0.0],
    Time    =   [0.0,0.0],
)
T.tmax[1]   =   9.886 * 1e6 * (60*60*24*365.25)   # [ s ]
nt          =   9999
# ------------------------------------------------------------------- #

To use **passive tracers for the advection of density, they first need to be initialized**.  
The function `IniTracer2D()` initializes the distribution of tracers in the given model domain (with optional random noise in their positions).  
If phases were previously defined in `Ini`, each tracer is assigned its corresponding phase.  

During the time loop, tracers are advected and transport the phase. With the function `Marker2Cells()`, a property of the phase (here the density ρ) is interpolated back onto the grid.  

If another advection method is chosen, the density is initialized directly on the grid.  
Note that the advection routines require information on the extended grid.  

To set up the initial conditions, we use the function `IniPhase`, which assigns each grid point to its corresponding phase. The density value associated with each phase is then mapped to the grid.  

In [ ]:
# Tracer Advection ================================================== #
if FD.Method.Adv==:tracers 
    # Tracer Initialization ---
    nmx,nmy     =   3,3
    noise       =   0
    nmark       =   nmx*nmy*NC.x*NC.y
    Aparam      =   :phase
    MPC         =   (
            c       =   zeros(Float64,(NC.x,NC.y)),
            v       =   zeros(Float64,(NV.x,NV.y)),
            th      =   zeros(Float64,(nthreads(),NC.x,NC.y)),
            thv     =   zeros(Float64,(nthreads(),NV.x,NV.y)),
    )
    MAVG        = (
            PC_th   =   [similar(D.wte) for _ = 1:nthreads()],  # per thread
            PV_th   =   [similar(D.ηv) for _ = 1:nthreads()],   # per thread
            wte_th  =   [similar(D.wte) for _ = 1:nthreads()],  # per thread
            wtv_th  =   [similar(D.wtv) for _ = 1:nthreads()],  # per thread
    )
    Ma      =   IniTracer2D(Aparam,nmx,nmy,Δ,M,NC,noise,Ini.p,phase)
    # RK4 weights ---
    rkw     =   1.0/6.0*[1.0 2.0 2.0 1.0]   # for averaging
    rkv     =   1.0/2.0*[1.0 1.0 2.0 2.0]   # for time stepping
    # Count marker per cell ---
    CountMPC(Ma,nmark,MPC,M,x,y,Δ,NC,NV,1)
    # Interpolate from markers to cell ---
    Markers2Cells(Ma,nmark,MAVG.PC_th,D.ρ_ex,MAVG.wte_th,D.wte,x,y,Δ,Aparam,ρ)
    D.ρ     .=  D.ρ_ex[2:end-1,2:end-1]  
else
    # --------------------------------------------------------------- #
    # Initial Condition ============================================= #
    # Phase ---
    # If tracers are used, phases need to be defined on the tracers 
    # directly and are not suppose to be interpolated from the centroids! 
    IniPhase!(Ini.p,D,M,x,y,NC;phase)
    for i in eachindex(phase)
        D.ρ[D.p.==phase[i]] .= ρ[i]
    end
    D.ρ_ex[2:end-1,2:end-1]     .=  D.ρ
    D.ρ_ex[1,:]     .=   D.ρ_ex[2,:]
    D.ρ_ex[end,:]   .=   D.ρ_ex[end-1,:]
    D.ρ_ex[:,1]     .=   D.ρ_ex[:,2]
    D.ρ_ex[:,end]   .=   D.ρ_ex[:,end-1]
    D.ρ_exo         .=   D.ρ_ex
end
# ------------------------------------------------------------------- #

Next, we need to define the parameters for the **linear system**.  

The numbering of the equations for the *x-component* and *y-component* of the momentum conservation, as well as for the *mass conservation*, together with the discretization of the equations using the finite difference method, is explained in detail [here](../../examples/StokesEquation/2D/README.md).  

In addition, the right-hand side vector `rhs` and the solution vector `χ` must be initialized.  
The size of both vectors is determined by the maximum number of equations.  

In [ ]:
# System of Equations =============================================== #
# Numbering, without ghost nodes! ---
off    = [  NV.x*NC.y,                          # vx
            NV.x*NC.y + NC.x*NV.y,              # vy
            NV.x*NC.y + NC.x*NV.y + NC.x*NC.y]  # Pt

Num    =    (
    Vx  =   reshape(1:NV.x*NC.y, NV.x, NC.y), 
    Vy  =   reshape(off[1]+1:off[1]+NC.x*NV.y, NC.x, NV.y), 
    Pt  =   reshape(off[2]+1:off[2]+NC.x*NC.y,NC...),
)
# ------------------------------------------------------------------- #

With the function `Assemblyc()`, the coefficients for the coefficient matrix $\mathbf{K}$ can be assigned.  
The function assigns the required coefficient values to each equation.  

Since we are dealing with an **isoviscous** problem, the coefficient matrix can be assembled **once outside the time loop**, as it does not change during the simulation.  

In [ ]:
# Assemble Coefficients ============================================= #
K       =   Assemblyc(NC, NV, Δ, η₀, VBC, Num)
# ------------------------------------------------------------------- #

Next comes the **time loop**. In each iteration, the following steps must be performed:

- Update the right-hand side (which contains the density distribution),  
- Solve the linear system,  
- Visualize the density distribution and velocity field,  
- Compute the time step size, and  
- Advect the density.  

In [ ]:
# Time Loop ========================================================= #
for it = 1:nt
    χ       =   zeros(maximum(Num.Pt))  #   Unknown Vector
    # Update Time ---
    T.Time[1]   =   T.Time[2] 
    @printf("Time step: #%04d, Time [Myr]: %04e\n ",it,
                T.Time[1]/(60*60*24*365.25)/1.0e6)
    # Momentum Equation ===
    # Update RHS ---
    rhs     =   updaterhsc( NC, NV, Δ, η₀, D.ρ, -g, VBC, Num )
    # Solve System of Equations ---
    χ       =   K \ rhs
    # Update Unknown Variables ---
    D.vx[:,2:end-1]     .=  χ[Num.Vx]
    D.vy[2:end-1,:]     .=  χ[Num.Vy]
    D.Pt                .=  χ[Num.Pt]
    # ===
    # Get the velocity on the centroids ---
    for i = 1:NC.x
        for j = 1:NC.y
            D.vxc[i,j]  = (D.vx[i,j+1] + D.vx[i+1,j+1])/2
            D.vyc[i,j]  = (D.vy[i+1,j] + D.vy[i+1,j+1])/2
        end
    end
    @. D.vc        = sqrt(D.vxc^2 + D.vyc^2)
    # ---
    @show(maximum(D.vc))
    @show(minimum(D.Pt))
    @show(maximum(D.Pt))
    # ---
    if T.Time[2] >= T.tmax[1]
        it = nt
    end
    # ---
    if mod(it,2) == 0 || it == nt || it == 1
        p = heatmap(x.c./1e3,y.c./1e3,D.ρ',color=:inferno,
                xlabel="x[km]",ylabel="y[km]",colorbar=false,
                title="Density",
                aspect_ratio=:equal,xlims=(M.xmin/1e3, M.xmax/1e3), 
                ylims=(M.ymin/1e3, M.ymax/1e3),
                layout=(2,2),subplot=1)
        quiver!(p,x.c2d[1:Pl.qinc:end,1:Pl.qinc:end]./1e3,
                y.c2d[1:Pl.qinc:end,1:Pl.qinc:end]./1e3,
                quiver=(D.vx[1:Pl.qinc:end,1:Pl.qinc:end].*Pl.qsc,
                        D.vyc[1:Pl.qinc:end,1:Pl.qinc:end].*Pl.qsc), 
                la=0.5,
                color="white",layout=(2,2),subplot=1)
        heatmap!(p,x.c./1e3,y.c./1e3,D.vxc',
                xlabel="x[km]",ylabel="y[km]",colorbar=false,
                title="V_x",color=cgrad(:batlow),
                aspect_ratio=:equal,xlims=(M.xmin/1e3, M.xmax/1e3),
                ylims=(M.ymin/1e3, M.ymax/1e3),
                layout=(2,2),subplot=3)
        heatmap!(p,x.c./1e3,y.c./1e3,D.vyc',
                xlabel="x[km]",ylabel="y[km]",colorbar=false,
                title="V_y",color=cgrad(:batlow),
                aspect_ratio=:equal,xlims=(M.xmin/1e3, M.xmax/1e3),
                ylims=(M.ymin/1e3, M.ymax/1e3),
                layout=(2,2),subplot=4)
        heatmap!(p,x.c./1e3,y.c./1e3,D.Pt',
                xlabel="x[km]",ylabel="y[km]",colorbar=false,
                title="P_t",color=cgrad(:lipari),
                aspect_ratio=:equal,xlims=(M.xmin/1e3, M.xmax/1e3),
                ylims=(M.ymin/1e3, M.ymax/1e3),
                layout=(2,2),subplot=2)
        if save_fig == 1
            Plots.frame(anim)
        elseif save_fig == 0
            display(p)
        end
    end
    if T.Time[2] >= T.tmax[1]
        break
    end
     # Calculate Time Stepping ---
    T.Δ[1]      =   T.Δfac * minimum((Δ.x,Δ.y)) / 
                        (sqrt(maximum(abs.(D.vx))^2 + maximum(abs.(D.vy))^2))
    @printf("\n")
    # Calculate Time ---
    T.Time[2]   =   T.Time[1] + T.Δ[1]
    if T.Time[2] > T.tmax[1] 
        T.Δ[1]      =   T.tmax[1] - T.Time[1]
        T.Time[2]   =   T.Time[1] + T.Δ[1]
    end
    # Advection ===
    if FD.Method.Adv==:upwind
        upwindc2D!(D.ρ,D.ρ_ex,D.vxc,D.vyc,NC,T.Δ[1],Δ.x,Δ.y)
    elseif FD.Method.Adv==:slf
        slfc2D!(D.ρ,D.ρ_ex,D.ρ_exo,D.vxc,D.vyc,NC,T.Δ[1],Δ.x,Δ.y)
    elseif FD.Method.Adv==:semilag
        semilagc2D!(D.ρ,D.ρ_ex,D.vxc,D.vyc,[],[],x,y,T.Δ[1])
    elseif FD.Method.Adv==:tracers
        # Advect tracers ---
        @printf("Running on %d thread(s)\n", nthreads())  
        AdvectTracer2D(Ma,nmark,D,x,y,T.Δ[1],Δ,NC,rkw,rkv,1)
        CountMPC(Ma,nmark,MPC,M,x,y,Δ,NC,NV,it)
        # Interpolate phase from tracers to grid ---
        Markers2Cells(Ma,nmark,MAVG.PC_th,D.ρ_ex,MAVG.wte_th,D.wte,x,y,Δ,Aparam,ρ)
        D.ρ     .=   D.ρ_ex[2:end-1,2:end-1]  
    end
end # End Time Loop

Finally, we save the animation to file for later inspection and analysis.  

In [ ]:
# Save Animation ==================================================== #
if save_fig == 1
    # Write the frames to a GIF file
    Plots.gif(anim, string( path, filename, ".gif" ), fps = 15)
    foreach(rm, filter(startswith(string(path,"00")), readdir(path,join=true)))
end